<a href="https://colab.research.google.com/github/vanyagoncharov/CourseMLResourse/blob/main/dz_23_s3e6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозировании цен на жилье в Париже

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

## Функции

## Загрузка данных

In [2]:
# Загружаем тестовый и тренировочный наборы данных
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/playground-series-s3e6/test.csv')
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/playground-series-s3e6/train.csv')

In [3]:
# Удаление ненужных столбцов, которые могут не способствовать прогнозированию модели.
data_test = data_test.drop(['id', 'cityCode'], axis=1)
data_train = data_train.drop(['id', 'cityCode'], axis=1)

In [4]:
data_test

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
0,47580,89,0,1,8,5,3,1995,0,0,6885,8181,241,0,8
1,62083,38,0,0,87,10,3,1994,1,1,4601,9237,393,1,4
2,90499,75,1,1,37,9,6,1997,0,1,7454,2680,305,0,2
3,16354,47,1,1,9,6,5,2019,1,1,705,5097,122,1,5
4,67510,8,0,0,55,3,7,2014,1,1,3715,7979,401,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,86184,74,1,1,73,2,10,1993,1,0,3714,3894,665,1,1
15150,45885,41,1,1,64,4,5,2008,0,0,783,7628,976,0,0
15151,73554,51,0,0,41,10,5,2004,0,1,8207,1777,775,0,10
15152,77604,51,0,1,33,4,3,2019,0,0,9578,9706,135,0,2


In [5]:
data_train

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,34291,24,1,0,47,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,55825,84,1,0,70,3,10,2000,0,0,4477,786,345,0,0,5594137.1
22726,65870,88,1,0,49,9,9,2015,0,1,4811,2454,755,0,7,6594705.0
22727,93192,42,1,0,39,10,5,2014,1,0,5595,4072,789,0,0,9321511.4
22728,65797,86,1,0,89,2,10,2000,1,0,5358,2513,411,0,0,6584708.2


## Разведочный анализ данных

### Общий анализ данных

In [6]:
pd.value_counts(data_train.dtypes)

int64      15
float64     1
dtype: int64

In [7]:
pd.value_counts(data_test.dtypes)

int64    15
dtype: int64

In [8]:
data_test = data_train.astype({"hasYard":"category","hasPool":"category","isNewBuilt":"category","hasStormProtector":"category","hasStorageRoom":"category"})
data_train = data_train.astype({"hasYard":"category","hasPool":"category","isNewBuilt":"category","hasStormProtector":"category","hasStorageRoom":"category"})

In [9]:
test_cat_cols = ["hasYard","hasPool","isNewBuilt","hasStormProtector","hasStorageRoom"]
train_cat_cols = ["hasYard","hasPool","isNewBuilt","hasStormProtector","hasStorageRoom"]

In [10]:
test_num_cols = ['squareMeters', 'numberOfRooms', 'floors', 'cityPartRange', 'numPrevOwners', "basement", "attic", "garage", "hasGuestRoom", "made"]
train_num_cols = ['squareMeters', 'numberOfRooms', 'floors', 'cityPartRange', 'numPrevOwners', "basement", "attic", "garage", "hasGuestRoom", "made"]

### Сколько строк в датафрейме, сколько столбцов

In [11]:
# Вывод информации о датасете
print(f'Информация о датасете: \n {data_train.shape[0]}: строк\n {data_train.shape[1]}: столбцов\n')

Информация о датасете: 
 22730: строк
 16: столбцов



### Сколько места занимает датафрейм в оперативной памяти

In [12]:
print(f'{np.round(data_train.memory_usage(index=True).sum() / 1024, 4)} КБ: занимаемый объем памяти')

2065.0762 КБ: занимаемый объем памяти


### Для каждой интервальной переменной подсчитать следующее - мин, медиана, среднее, макс и персентили 25, 75

In [13]:
data_train[train_num_cols].describe()

,squareMeters,numberOfRooms,floors,cityPartRange,numPrevOwners,basement,attic,garage,hasGuestRoom,made
count,2.273000e+04,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000
mean,4.658622e+04,48.241091,47.305983,5.585042,5.620766,5283.646634,5067.981698,530.469644,5.153058,2008.005059
std,4.952124e+04,28.226428,47.777207,2.739533,2.713026,3047.084412,3097.347939,274.840604,3.055246,118.826777
min,8.900000e+01,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,4.000000,0.000000,1990.000000
25%,2.039275e+04,25.000000,25.000000,3.000000,3.000000,2977.250000,2599.000000,297.000000,3.000000,2000.000000
50%,4.448400e+04,47.000000,45.000000,6.000000,6.000000,5359.000000,4977.000000,515.000000,5.000000,2006.000000
75%,7.154700e+04,75.000000,69.000000,8.000000,8.000000,7746.000000,7652.000000,767.000000,8.000000,2014.000000
max,6.071330e+06,100.000000,6000.000000,10.000000,10.000000,91992.000000,96381.000000,9017.000000,10.000000,10000.000000


### **Для каждой категориальной переменной рассчитать моду и сколько раз мода встречается в данных!!!!!!!!!!!**

In [14]:
train_cat_data = data_train[train_cat_cols]

# Рассчитываем моду и количество ее встреч для каждой переменной
mode_df = pd.DataFrame({'mode': train_cat_data.mode().iloc[0], 'count': train_cat_data.apply(lambda x: x.value_counts().max())})

# Выводим результат
print(mode_df)

                  mode  count
hasYard              0  11913
hasPool              0  12439
isNewBuilt           0  12093
hasStormProtector    0  12274
hasStorageRoom       0  12236


## Подготовка датасета к построению моделей ML

### Провести анализ и обработку пропусков (либо заменить, либо удалить)

In [15]:
data_test.isnull().sum()

squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
price                0
dtype: int64

In [16]:
data_train.isnull().sum()

squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
price                0
dtype: int64

Пропуски в датасете отсутствуют

### **Провести анализ и обработку выбросов (либо заменить, либо удалить)**


https://www.kaggle.com/code/polaninethajiram/house-price-regression

### **Провести анализ и обработку категориальных переменных (сколько таких переменных, закодируйте категориальные переменные одним из методов ( one hot encoding, mean target, frequence encoding)**

Исходя из анализа выше было видно, что категориальные переменные отсутствуют

### Разделить датасет на трейн и тест

In [17]:
# Разделение данные на целевую переменную и признаки
X = data_train.iloc[:, :-1]
y = data_train.iloc[:, -1]

In [18]:
# Разделение данных на обучающую выборку и валидационную выборку
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [19]:
# Стандартизация (нормализация) числовых признаков
scaler = StandardScaler()
scaler = scaler.fit(X_train[train_num_cols])
X_train_std = pd.DataFrame(scaler.transform(X_train[train_num_cols]), columns=X_train[train_num_cols].columns)
X_val_std = pd.DataFrame(scaler.transform(X_val[train_num_cols]), columns=X_val[train_num_cols].columns)

In [20]:
# Преобразование категориальных признаков в бинарные (dummy)
X_train_dum = pd.get_dummies((X_train[train_cat_cols]), drop_first=True)
X_val_dum = pd.get_dummies((X_val[train_cat_cols]), drop_first=True)

In [21]:
# Сброс индексов
X_train_std = X_train_std.reset_index()
X_train_std = X_train_std.drop(["index"], axis=1)

X_train_dum = X_train_dum.reset_index()
X_train_dum = X_train_dum.drop(["index"], axis=1)

X_val_std = X_val_std.reset_index()
X_val_std = X_val_std.drop(["index"], axis=1)

X_val_dum = X_val_dum.reset_index()
X_val_dum = X_val_dum.drop(["index"], axis=1)

In [22]:
# Объединение стандартизированных числовых признаков и бинарных переменных для категориальных признаков в обучающей и валидационной выборках.
final_X_train = pd.concat([X_train_std, X_train_dum], axis=1)
final_X_val = pd.concat([X_val_std, X_val_dum], axis=1)

In [30]:
y_train

9111     8225077.0
21095    9885562.4
7425     3589966.3
12715    1351303.0
2145     3850352.1
           ...    
15377    6272944.7
21602    5153610.4
17730    2086273.0
15725    6089911.5
19966    4759722.6
Name: price, Length: 18184, dtype: float64

In [23]:
# Проверка на пустые значения
final_X_train.isnull().sum()

squareMeters           0
numberOfRooms          0
floors                 0
cityPartRange          0
numPrevOwners          0
basement               0
attic                  0
garage                 0
hasGuestRoom           0
made                   0
hasYard_1              0
hasPool_1              0
isNewBuilt_1           0
hasStormProtector_1    0
hasStorageRoom_1       0
dtype: int64

In [24]:
# Проверка на пустые значения
final_X_val.isnull().sum()

squareMeters           0
numberOfRooms          0
floors                 0
cityPartRange          0
numPrevOwners          0
basement               0
attic                  0
garage                 0
hasGuestRoom           0
made                   0
hasYard_1              0
hasPool_1              0
isNewBuilt_1           0
hasStormProtector_1    0
hasStorageRoom_1       0
dtype: int64

## Обучить любые 2 алгоритма:

### SVM

### Decision tree

In [25]:
dt_classifier = DecisionTreeClassifier()

In [26]:
# Определение сетки параметров для GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [27]:
# Создаем объект GridSearchCV
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

In [28]:
# Обучаем GridSearch на обучающей выборке
grid_search.fit(final_X_train, y_train)

ValueError: ignored

In [ ]:
# Выводим наилучшие параметры и точность на тестовой выборке
print("Наилучшие параметры: ", grid_search.best_params_)
print("Точность на тестовой выборке: ", grid_search.best_score_)

## Оценить качество алгоритмов, выбрать самый оптимальный алгоритм

Затем сравните их производительность, используя метрики, такие как MAE, MSE, R^2, на тестовых данных. Обоснуйте свой выбор лучшей модели, учитывая точность
прогнозирования и обобщающую способность на новых данных